# Import Necessary Modules #

In [ ]:
#Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.ticker as mtick
import mglearn
from scipy.optimize import curve_fit
from numpy import unique
import numpy as np
from numpy import where
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
#%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.stats import zscore
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
from factor_analyzer.factor_analyzer import calculate_kmo
from IPython.display import display, HTML

# Data Cleaning and Processing #

In [ ]:
#Display of column and row
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.max_rows = 10 # You can set the number of row you want to see.
#Read dataset
df_main_files=pd.read_csv('platee.csv')
df_main_files.columns
#choose the relevant dataset column needed
dfiles=df_main_files[['plug_id','field_name', 'sw', 'pc', 'kl','porosity', 'surface_tension', 'cos_contact_angle' ]]
dfiles
## Define a new subset for the needed parameters and remove all negative value to acheive a positive value only##
d_newfiles=dfiles[['plug_id', 'sw', 'pc', 'kl','porosity','surface_tension', 'cos_contact_angle']]
d_nonegative=d_newfiles[d_newfiles>= 0].dropna()
d_nonegative
#drop_duplicates('plug_id')


In [ ]:

# Check for duplicates in files
dl=d_nonegative
d_checkcolumfornumberofid= dl.copy().drop_duplicates('plug_id')
d_checkcolumfornumberofid
#dl.describe()


## This code plot all the capillary pressure from our dataset before extractting the primary drainge capilary pressure.#

In [ ]:
# This code plot all the capillary pressure from our dataset before extractting the primary drainge capilary pressure.
#loop through each unique plug_id in the column and plot the values##

plt.figure(figsize=(10,6))
#This loops over each unique value and returns an array of all unique values
for plug_id in dl['plug_id'].unique():
#For the current plug_id in the loop, filters the DataFrame to create a subset containing only rows where plug_id matches the current value.
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['sw'], subset['pc'])

# plot of Capillary pressure against Water Saturation 

plt.xlabel('Water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc)')
plt.title('Capillary Pressure Curve for all plug_ids')
plt.grid()
#plt.yscale('log')
plt.ylim(-2,16)
#plt.xlim(0, 1.05)
plt.savefig('TheFirstccapillarycurve.png')
#plt.yscale('log')
plt.show()

In [ ]:
# This code Check for any plug Id to view the PC and Sw value sw cut off ##

first_dz=dl.copy()
d_checkprimarydrainage= first_dz.copy().drop_duplicates('plug_id')
first_dz
d_justcheck1=first_dz.query('plug_id in [6558]')
d_justcheck1
#d_checkprimarydrainage

## This code is for extracting the primary drainge curve it works by Initiating cut Off Value for where Sw value start increasing ##

In [ ]:
## Cut off values where sw start to increase after a steady decrease in Sw Values #

df_comb = first_dz.copy()

# cut off values where 'sw' starts increasing after a steady decrease

def sw_cut_off_values(group):
    if (group['pc'] < 0).any():
        # If any 'pc' value is negative, drop the entire group
        return pd.DataFrame()
    # Point where 'SW' starts to increase after a steady decrease
    start_decrease = (group['sw'] > group['sw'].shift(1))
    idx_increase_start = start_decrease[start_decrease].index.min()

    # Cut off values at the identified index
    group.loc[idx_increase_start:, 'sw'] = float('nan')

    return group

# Apply the function to each group of 'id_num'
df_result = df_comb.groupby('plug_id', group_keys=True).apply(sw_cut_off_values)

# Drop rows with NaN values in 'SW'
df_result = df_result.dropna(subset=['sw'])

#Print modified DataFrame
print(df_result)
df_result.to_csv('testing.csv', index=False)



In [ ]:
#Read new file after sorting and check for any plug ID after cut off #
# Sample of new primary drainage
df_cuttoff_data=pd.read_csv('testing.csv')
df_cuttoff_data.dropna()
justcheck=df_cuttoff_data.query('plug_id in [6558]')
justcheck


#df_result

In [ ]:
##Plot the new file after sorting ##
# Primary Drianage Curve
plt.figure(figsize=(10,6))
for plug_id in df_cuttoff_data['plug_id'].unique():
    subset = df_cuttoff_data[df_cuttoff_data['plug_id'] == plug_id]
    plt.plot(subset['sw'], subset['pc'])

#plot Pc vs Sw
plt.xlabel('Water saturation (Sw)')
plt.ylabel('Capillary Pressure (Pc)')
plt.title('Primary Drainage Capillary Pressure vs Water Saturation')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
plt.savefig('Onlyprimarydrainagecapillary_curve.png')
#plt.yscale('log')
plt.show()

In [ ]:
d_justcheck1.query

In [ ]:

single_test2=dl
unique_plug_id4=single_test2[ 'plug_id'].unique()
for plug_id in unique_plug_id4:
    subset2 = single_test2[single_test2['plug_id'] == plug_id]
    pc_curve = subset2['pc']
    sw_curve = subset2['sw']
    plt.scatter(sw_curve, pc_curve, label=f'plug_id plot {plug_id}')
    #plt.plot(sw_curve, pc_curve, label=f'plug_id plot {plug_id}')
plt.xlabel('Water Saturation, Sw')
plt.ylabel('Capillary Pressure, Pc'),
plt.title(f'PC vs Sw for single plug_id {plug_id} before extraction')
    #plt.legend()
plt.grid()
    #plt.yscale('log')
    #plt.savefig('Figurebeforecutoff.png')
plt.show()

In [ ]:

single_test2=dl.query('plug_id in [6558]')
unique_plug_id4=single_test2[ 'plug_id'].unique()
for plug_id in unique_plug_id4:
    subset2 = single_test2[single_test2['plug_id'] == plug_id]
    pc_curve = subset2['pc']
    sw_curve = subset2['sw']
    plt.scatter(sw_curve, pc_curve, label=f'plug_id plot {plug_id}')
    plt.plot(sw_curve, pc_curve)
    plt.xlabel('Water Saturation, Sw')
    plt.ylabel('Capillary Pressure, Pc'),
    plt.title(f'PC vs Sw for sample plug_id before extraction')
    #plt.legend()
    plt.grid()
    #plt.yscale('log')
    plt.savefig('Figurebeforecutoff.png')
    plt.show()

In [ ]:
#Visualize data for a single plug ID 
single_test=df_cuttoff_data.query('plug_id in [6558]')
unique_plug_id2=single_test[ 'plug_id'].unique()
for plug_id in unique_plug_id2:
    subset2 = single_test[single_test['plug_id'] == plug_id]
    pc_curve = subset2['pc']
    
    sw_curve = subset2['sw']
    plt.scatter(sw_curve, pc_curve, label=f'plug_id plot {plug_id}')
    plt.plot(sw_curve, pc_curve)
    plt.xlabel('Water Saturation, Sw')
    plt.ylabel('Capillary Pressure, Pc'),
    plt.title(f'PC vs Sw for sample plug_id after extraction')
    #plt.legend()
    plt.grid()
    #plt.yscale('log')
    plt.savefig('Figureaftercuttoff.png')
    plt.show()

In [ ]:
Check_beforecut= dl.copy()
# unique plug_id values
unique_plug_id9= Check_beforecut['plug_id'].unique()

# Extract and plot PC and Sw for each unique plug_ids
for plug_id in unique_plug_id9:
    subset1 = Check_beforecut[Check_beforecut['plug_id'] == plug_id]
#plot the whole PC curve from the dataset 
    pc_curve = subset1['pc']
    sw_curve = subset1['sw']
    
    plt.plot(sw_curve, pc_curve, label=f'plug_id {plug_id}')
    plt.xlabel('Water Saturation, (Sw )')
    plt.ylabel('Capillary Pressure, (Pc)')
    plt.title(f'PC vs Sw for plug_id {plug_id} before PD extraction')
    plt.legend()
    plt.grid()
    #plt.savefig(f'plug_id_{plug_id}_curvesss.png')
    plt.show(2)

In [ ]:
#visualize data and plot separate chart for each plug ID

# unique plug_id values
unique_plug_id1= df_cuttoff_data['plug_id'].unique()

# Extract and plot PC and Sw for each unique plug_ids
for plug_id in unique_plug_id1:
    subset1 = df_cuttoff_data[df_cuttoff_data['plug_id'] == plug_id]
#plot the whole PC curve from the dataset 
    pc_curve = subset1['pc']
    sw_curve = subset1['sw']
    
    plt.plot(sw_curve, pc_curve, label=f'plug_id {plug_id}')
    plt.xlabel('Water Saturation, (Sw )')
    plt.ylabel('Capillary Pressure, (Pc)')
    plt.title(f'PC vs Sw for plug_id {plug_id} after PD extraction')
    plt.legend()
    plt.grid()
    #plt.savefig(f'plug_id_{plug_id}_curvesss.png')
    plt.show(2)

In [ ]:
#histogram chart for priority variables
df_cuttoff_data.hist(bins=60,figsize=(20,10))
plt.show()


 
# #Modeling the data using Skjaevland primary drainage model.


# Non-linear least squares optimization method.

This was used to estimate the unkown parameters of the Skjaeveland model.

Algorithm Used: Trust Region Reflective Algorithm this was due to the specified bounds 

### Optimization of swirr, c and a parmaters and plotting each plug separately

In [ ]:

'''''''''''''''
# load the data  
df = df_cuttoff_data

# Skjæveland model function
def skjaevland_model(sw, swirr, c, a):
    return c / ((sw - swirr) / (1 - swirr)) ** a

# Initial guesses for the parameters
#initial_guess = [0.05, 0.2, 2.0]  # see [swirr, c, a]

# Bounds for the parameters
#param_bounds = ([0.0, 0.0, 0.0], [5, np.inf, np.inf])

# Group the data by plug_id
grouped = df.groupby('plug_id')

# Dictionary to store results
results = {}

# Loop through each plug_id group and perform curve fitting
for plug_id, group in grouped:
    sw_data = group['sw'].values
    pc_data = group['pc'].values
    # Sort data by sw to improve fitting stability
    sort_idx = np.argsort(sw_data)
    sw_data = sw_data[sort_idx]
    pc_data = pc_data[sort_idx]
    if sw_data[0]==1.0 and pc_data[0]==0:
        sw_data=sw_data[1:]
        pc_data=pc_data[1:]
    if np.size(sw_data)>2:
        
        initial_guess = [np.min(sw_data)*0.8, 0.1, 0.6]  # [swirr, c, a]
        param_bounds = ([0.0, 0.0, 0.0], [np.min(sw_data), np.inf, np.inf])
    

   # Code to Perform curve fitting
        try:
            popt, pcov = curve_fit(
                skjaevland_model, sw_data, pc_data,
                p0=initial_guess, bounds=param_bounds, maxfev=10000
            )
            swirr_opt, c_opt, a_opt = popt
            results[plug_id] = {'swirr': swirr_opt, 'c': c_opt, 'a': a_opt}

            # Print out the optimized parameters
            print(f"plug_id: {plug_id}")
            print(f"  Optimized Swirr: {swirr_opt:.6f}")
            print(f"  Optimized c: {c_opt:.6f}")
            print(f"  Optimized a: {a_opt:.6f}")

            # Plot the data against the fitted model
            sw_fit = np.linspace(min(sw_data), max(sw_data), 100)
            pc_fitted = skjaevland_model(sw_fit, swirr_opt, c_opt, a_opt)

            plt.figure() 
            plt.scatter(sw_data, pc_data, label='Experimental Data', color='blue', alpha=0.6)
            plt.plot(sw_fit, pc_fitted, label='Fitted Model', color='red')
            plt.xlabel('Water Saturation (Sw)')
            plt.xlim(0,1.02)
            plt.ylim(-0.1, np.max(pc_data) )
            plt.ylabel('Capillary Pressure (Pc)')
            plt.title(f'Experimental Data Vs Fitted Model: {plug_id}')
            plt.legend()
            plt.grid(True)
            plt.show()


    
        except RuntimeError as e:
            print(f"Fitting failed for plug_id: {plug_id}. Error: {e}")
            
    else:
        print('Plotting data less than two for this plug_id, provide more')

plt.plot(df_main_files['pc'], df_main_files['sw'])

plt.xlabel('Water saturation (Sw)')
plt.ylabel('Capillary Pressure (Pc)')
plt.title('Primary Drainagee Capillary Pressure vs Water Saturation')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
#plt.savefig('Onlyprimarydrainagecapillary_curve.png')
#plt.yscale('log')
plt.show()


## Non-linear least squares optimization method.

### Optimization of Swirr, C and a parmaters and plotting of all plug chart into one.

# Non-linear least squares optimization method.

## Optimization of Swirr, C and a parmaters and plotting each plug in one chart.

### Return value with goodness of fit ( Rsquared, Rsme and Mae)

### Inttroduced a cut off value using the goodness of fit (Rsquared)

In [ ]:

# Read the data 
df = df_cuttoff_data

# Define the Skjæveland model function
def skjaevland_model(sw, swirr, c, a):
    return c / ((sw - swirr) / (1 - swirr)) ** a
    

# Initial guesses for the parameters
#initial_guess = [0.05, 0.2, 2.0]  # [swirr, c, a]

# Bounds for the parameters
#param_bounds = ([0.0, 0.0, 0.0], [0.5, np.inf, np.inf])

# Group the data by plug_id
grouped = df.groupby('plug_id')

# Initialize dictionaries to store results and R-squared values
results = {}
r2_values = {}

# Prepare a list to store all data for CSV export
csv_data = []
fit_data = []
# Set the R-squared threshold
r2_threshold = 0.96

# Set up the plot for filtered plugs
plt.figure(figsize=(12, 8))

# Color and marker map
colors = plt.cm.tab20.colors  # Use a colormap with enough distinct colors
markers = ['o', 's', 'v', '^', '<', '>', 'P', 'X', 'D', '*', 'h', '+', 'x', 'd', '|', '_']

# Index for plotting colors and markers
plot_idx = 0

# Loop through each plug_id group and perform curve fitting
for idx, (plug_id, group) in enumerate(grouped):
    sw_data = group['sw'].values
    pc_data = group['pc'].values
    kl_data = group['kl'].values
    porosity_data = group['porosity'].values
    surface_data= group['surface_tension'].values
    contact_data= group['cos_contact_angle'].values

    # Sort data by sw to improve fitting stability
    sort_idx = np.argsort(sw_data)
    sw_data = sw_data[sort_idx]
    pc_data = pc_data[sort_idx]
    kl_data = kl_data[sort_idx]
    porosity_data = porosity_data[sort_idx]
    surface_data = surface_data[sort_idx]
    
 #Remove unwanted values in the dataset  
    if sw_data[0]==1.0 and pc_data[0]==0:
        sw_data=sw_data[1:]
        pc_data=pc_data[1:]
 # Check if the size of the array or number of element in Sw_data is greater that 2         
    if np.size(sw_data)>2:
#provide guesses and bounds for the optimization process         
        initial_guess = [np.min(sw_data)*0.8, 0.1, 0.6]  # [swirr, c, a]
        param_bounds = ([0.0, 0.0, 0.0], [np.min(sw_data), np.inf, np.inf])

        # Perform curve fitting
        try:
            popt, pcov = curve_fit(
                skjaevland_model, sw_data, pc_data,
                p0=initial_guess, bounds=param_bounds, maxfev=10000
            )
            swirr_opt, c_opt, a_opt = popt
            results[plug_id] = {'swirr': swirr_opt, 'c': c_opt, 'a': a_opt}
            np.linalg.cond(pcov)

            # perform goodness-of-fit metrics
            pc_pred = skjaevland_model(sw_data, swirr_opt, c_opt, a_opt)
            r2 = r2_score(pc_data, pc_pred)
            rmse = np.sqrt(mean_squared_error(pc_data, pc_pred))
            mae = mean_absolute_error(pc_data, pc_pred)
            sw_mini= np.min(sw_data)

            # Store the metrics
            results[plug_id].update({'R2': r2, 'RMSE': rmse, 'MAE': mae})
            r2_values[plug_id] = r2

      
            # Print the optimized parameters with the metrics
            print(f"Plug ID: {plug_id}")
            print(f"  Optimized Parameters:")
            print(f"    Swirr = {swirr_opt:.6f}")
            print(f"    c     = {c_opt:.6f}")
            print(f"    a     = {a_opt:.6f}")
            print(f"  Goodness-of-Fit Metrics:")
            print(f"    R-squared = {r2:.4f}")
            print(f"    RMSE      = {rmse:.6f}")
            print(f"    MAE       = {mae:.6f}\n")

            # Filtering based on R-squared- G
            if r2 >= r2_threshold:
                # Generate sw values for plotting the fitted model
                sw_fit = np.linspace(min(sw_data), max(sw_data), 100)
                pc_fitted = skjaevland_model(sw_fit, swirr_opt, c_opt, a_opt)

                # Select color and marker
                color = colors[plot_idx % len(colors)]
                marker = markers[plot_idx % len(markers)]
                plot_idx += 1

                # Plot the data points
                #plt.scatter(sw_data, pc_data, label=f'Plug {plug_id} Data', color=color, marker=marker, alpha=0.6)

                # Plot the fitted model
                plt.plot(sw_fit, pc_fitted, label=f'Plug {plug_id} Fit', color=color)

              # Append data to csv_data list
            for sw_val, pc_val, kl_val, por_val, sur_val,cos_val in zip(sw_data, pc_data, kl_data, porosity_data,surface_data, contact_data):
                csv_data.append({
                    'plug_id': plug_id,
                    'sw': sw_val,
                    'pc': pc_val,
                    'kl': kl_val,
                    'porosity': por_val,
                    'surface_tension' : sur_val,
                    'cos_contact_angle': cos_val,
                    'swirr': swirr_opt,
                    'c': c_opt,
                    'a': a_opt,
                    'R2': r2,
                    'sw_mini' : sw_mini,
                    'RMSE': rmse,
                    'MAE': mae
                    
                })
                
              # Append fitted curve data to fitted data list
          
            for swf, pcf in zip(sw_fit, pc_fitted):
                fit_data.append({
                    'plug_id': plug_id,
                    'sw_fit': swf,
                    'pc_fitted': pcf,
                    'kl': np.nan,          # No kl value for 
                    'porosity': np.nan,  # No porosity value 
                    'swirr': swirr_opt,
                    'c': c_opt,
                    'a': a_opt,
                    'R2': r2,
                    'RMSE': rmse,
                    'MAE': mae,
                    'type': 'fit'   
                })

        except RuntimeError as e:
            print(f"Fit failed for Plug ID: {plug_id}. Error: {e}")
            
    else:
        print('Plot data less than two for this plug_id, please provide more data')

# Customize the plot for filtered plugs
plt.xlabel('Water Saturation (Sw)', fontsize=12)
plt.ylabel('Capillary Pressure (Pc)', fontsize=12)
plt.title(f'Modelled Capillary Pressure Curve with $R^2 \\geq {r2_threshold}$', fontsize=14)
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize='small')
plt.grid(True)
plt.tight_layout()
plt.savefig('Representative Capillary Pressure for SKJA')
plt.show()


In [ ]:
# Visualizations

#Plot R2 Value

sorted_plug_ids = sorted(r2_values.keys())
sorted_r2_values = [r2_values[pid] for pid in sorted_plug_ids]

# Scatter Plot with Seaborn
plt.figure(figsize=(12, 8  ))
data_r2 = pd.DataFrame({'plug_ids': sorted_plug_ids,  'R2_values': sorted_r2_values})

# Create a column to distinguish above and below threshold
data_r2['Threshold Status'] = np.where(data_r2['R2_values'] >= r2_threshold, 'Above Threshold', 'Below Threshold')

# Seaborn scatterplot with hue for color distinction
sns.scatterplot(data=data_r2, x='plug_ids', y='R2_values', hue='Threshold Status', palette={'Above Threshold':'blue', 'Below Threshold': 'salmon'}, s=100, edgecolor='black')

# Plot the threshold line
plt.axhline(r2_threshold, color='red', linestyle='--', linewidth=2, label=f'$R^2$ Threshold = {r2_threshold}')

plt.xlabel('Plug ID', fontsize=12)
plt.ylabel('$R^2$ Value', fontsize=12)
plt.title('($R^2$ Values) for All Plugs ', fontsize=14)
plt.xticks(rotation=45)
plt.ylim(0, 1.05)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('RepresentativeR2 value ')
plt.show()

# Histogram of R-squared Values
plt.figure(figsize=(10, 6))
plt.hist(sorted_r2_values, bins=10, color='skyblue', edgecolor='black', alpha=0.7)
plt.axvline(r2_threshold, color='red', linestyle='--', linewidth=2, label=f'$R^2$ Threshold = {r2_threshold}')
plt.xlabel('$R^2$ Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of $R^2$ Values Across All Plugs ', fontsize=14)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.gca().xaxis.set_major_formatter(mtick.PercentFormatter(1.0))
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('Representativefrequencyditr ')
plt.show()

# Export data to CSV file
csv_df = pd.DataFrame(csv_data)
csv_df.to_csv('not_fitted.csv', index=False)
print("CSV file 'not_fitted.csv' has been saved.")

# Export fitted curves to a separate CSV file
fit_df = pd.DataFrame(fit_data)
fit_df.to_csv('fitted_curves.csv', index=False)
print("CSV file 'fitted_curves.csv' has been saved.")

In [ ]:
# For Non Pc Fitted curve
dc_merge= pd.read_csv('not_fitted.csv')
dc_merge

In [ ]:
# For Fitted Pc Curve 

filter_merge= pd.read_csv('fitted_curves.csv')
filter_merge
filter_merge.describe()

## Calculating varibles for Correlation and Clustering

In [ ]:

# Calculate and return variable

dc_merge['scaled_sw']=(dc_merge['sw']- dc_merge['swirr'])/(1-dc_merge['swirr'])
dc_merge['RQI']=(0.0314*np.sqrt((dc_merge['kl'])/dc_merge['porosity']))
dc_merge['J_function']= (dc_merge['pc'])/(dc_merge['surface_tension']*dc_merge['cos_contact_angle'])* (np.sqrt((dc_merge['kl'])/dc_merge['porosity']))
dc_merge['Norm_porosity']=(dc_merge['porosity'])/(1-dc_merge['porosity'])
dc_merge['FZI']= dc_merge['RQI']/dc_merge['Norm_porosity']
dc_merge['Radius']= 2*(dc_merge['surface_tension'])*dc_merge['cos_contact_angle']/(dc_merge['pc'])
dc_merge['logkl']= np.log10(dc_merge['kl'])
dc_merge['poro_log']= np.log10(dc_merge['porosity'])
dc_merge['c_log']= np.log10(dc_merge['c'])
dc_merge['a_log']= np.log10(dc_merge['a'])
dc_merge['swmini_log']= np.log10(dc_merge['sw_mini'])
dc_merge['swirr_log']= np.log10(dc_merge['swirr'])

dc_merge

In [ ]:
''''
df_csv1 = pd.DataFrame(dc_merge)
df_csv1.to_csv('good_fit.csv', index=False)
print("CSV file 'good_fit.csv' has been saved.")

In [ ]:
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))

sns.scatterplot(dc_merge, y="swirr", x="sw_mini" ,palette= 'magma', s=90 ,markers=True)
plt.xlabel('Lowest water saturation')
plt.ylabel('Irreducible water saturation')
plt.title( 'Irreducible water saturation vs Lowest water saturation')
#plt.ylim(0.024,5)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
plt.savefig('SWVSSWMINI.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:
dist_forsw=dc_merge[['c']]

dist_forsw.hist(bins=10,figsize=(20,10),xlabelsize=20, ylabelsize=18) 
#plt.title( 'Pore size distribution (a)',fontsize=20) 
plt.title( 'Entry pressure (c)',fontsize=29) 
plt.savefig('Distributgionofcswirr')
plt.show()
#dist_forswe=dc_merge[['c']]


#plt.title( 'Entry pressure (c)',fontsize=29) 
#plt.savefig('Distributgionofcswirr')
#plt.show()



In [ ]:
#Plotting of fitted Pc vs Sw using the Skjaeveland primary drainage model

plt.figure(figsize=(10,6))
sns.set_theme(style="ticks")
palette = sns.color_palette("tab20", n_colors=264)
sns.lineplot(filter_merge, y="pc_fitted", x="sw_fit",hue="plug_id", palette=palette ,markers=False, dashes=False, legend=False)
plt.xlabel('Water saturation')
plt.ylabel('Capillary Pressure (pc)')
plt.title('Modelled Primary Drainage Capillary Pressure Curve ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
#plt.savefig('newpc curve.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:
# Pc vs Sw from the Original Data which is not fitted or modelled
#Water saturation here is scaled.

plt.figure(figsize=(10,6))
sns.set_theme(style="ticks")
palette = sns.color_palette("tab20", n_colors=264)
sns.lineplot(dc_merge, y="pc", x="scaled_sw",hue="plug_id", palette=palette ,markers=False, dashes=False, legend=False)
plt.xlabel('Water saturation')
plt.ylabel('Capillary Pressure(Pc)')
plt.title('Primary Drainage Capillary Pressure Curve ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)SS
#plt.legend()
plt.grid()
plt.savefig('Thenewprimarydrainagecapillarycurve.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:

# J- Fuction Curve -- levrett J-function curve.


plt.figure(figsize=(10,6))
sns.set_theme(style="ticks")
palette = sns.color_palette("tab20", n_colors=264)
sns.lineplot(dc_merge, x="scaled_sw", y="J_function", hue="plug_id", palette=palette ,markers=False, dashes=False, legend=False)
#sns.scatterplot(dc_merge, x="sw", y="J_function", hue="plug_id" , palette="coolwarm" ,markers=True)

plt.xlabel('Scaled (Sw) ')
plt.ylabel('J-function, (Sw)')
plt.ylim(0,10)
plt.title(f'J-function curve for all Plug_ids')
#plt.legend()
plt.grid()
plt.savefig('Jfunction curve.png')
#plt.yscale('log')
plt.show()

In [ ]:
# J- fuction curve with joint plot 
plt.figure(figsize=(16,12))
#sns.scatterplot(data=dp_data, x="porosity",y="kl", hue="plug_id", palette="coolwarm")
sns.jointplot(dc_merge, x="scaled_sw", y="J_function", hue="plug_id" , palette="coolwarm" ,markers=True, legend=False)
#sns.scatterplot(a, x="sw", y="j_function", hue="plug_id" , palette="coolwarm" ,markers=True)

plt.title(f'J_function (sw)')
plt.xlabel('Scaled, Sw')
plt.ylabel('J-Function, J(sw)')
#plt.legend()
plt.grid()
plt.savefig('J - Function.png')
#plt.yscale('log')
plt.show()

In [ ]:
# Implot for Irreducible water saturation vs Log permeability

plt.figure(figsize=(8,6))
#sns.scatterplot(data=dp_data, x="porosity",y="kl", hue="plug_id", palette="coolwarm")
#sns.scatterplot(dc_merge, x="swirr", y="logkl", palette="coolwarm" ,markers=True, legend=False)
sns.lmplot(dc_merge, x="swirr", y="logkl" ,palette=palette, lowess=True, line_kws={"color": "C3"})
#sns.scatterplot(a, x="sw", y="j_function", hue="plug_id" , palette="coolwarm" ,markers=True)

plt.title(f'Log peremability vs Irreducible water saturation')
plt.xlabel('Irreducible water saturation (Sw)')
plt.ylabel('Permeability (md)')
#plt.legend()
plt.grid()
plt.savefig('Log Peremability vs Irreducible water saturation.png')
#plt.yscale('log')
plt.show()

In [ ]:
# Scatter plot for Permeabiltiy Vs irreducible water saturation

sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#palette = sns.color_palette("tab20", n_colors=263)
sns.scatterplot(x="swirr", y="logkl", data=dc_merge);
#sns.lmplot(dc_merge, x="porosity", y="logkl" ,palette=palette, lowess=True, line_kws={"color": "C3"})
#sns.scatterplot(a, y="kl", x="porosity" ,hue='plug_id', palette="coolwarm" ,markers=True)
plt.title(f'Log peremability vs Irreducible water saturation')
plt.xlabel('Irreducible water saturation (Sw)')
plt.ylabel('Permeability (md)')
#plt.legend()
plt.grid()
plt.savefig('Log Peremability vs Irreducible water saturation.png')
#plt.yscale('log')
plt.show()

In [ ]:
# Log permeability vs irredicble water saturationm

sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#palette = sns.color_palette("tab20", n_colors=263)
sns.scatterplot(y="c", x="RQI", data=dc_merge);
#sns.lmplot(dc_merge, x="porosity", y="logkl" ,palette=palette, lowess=True, line_kws={"color": "C3"})
#sns.scatterplot(a, y="kl", x="porosity" ,hue='plug_id', palette="coolwarm" ,markers=True)
plt.title(f'Log peremability vs Irreducible water saturation')
plt.xlabel('Irreducible water saturation (Sw)')
plt.ylabel('Permeability (md)')
#plt.legend()
plt.grid()
plt.savefig('Log Peremabilityg vs Irreducible water saturation.png')
#plt.yscale('log')
plt.show()

In [ ]:
#Permeabiltiy Vs Porosity using Joint plot

sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#palette = sns.color_palette("tab20", n_colors=263)
#sns.lmplot(x="porosity", y="logkl", data=dc_merge,  
    #        robust=True, scatter_kws={"s": 80});
#x_estimator=np.mean
#x_jitter=.05
sns.jointplot(x="porosity", y="logkl", data=dc_merge, kind="reg")
#sns.lmplot(dc_merge, x="porosity", y="logkl" ,palette=palette, lowess=True, line_kws={"color": "C3"})
#sns.scatterplot(a, y="kl", x="porosity" ,hue='plug_id', palette="coolwarm" ,markers=True)
plt.xlabel('Porosity(frac)')
plt.ylabel('Permeability(md)')
plt.title(f'Joint plot of Permeability vs Porosity ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
plt.legend()
plt.grid()
plt.savefig('jointklpror.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:
dc_merge

In [ ]:
df_check=dc_merge[['kl','logkl','porosity','poro_log','c','c_log','J_function', 'a_log' , 'a','swirr' ,'RQI','sw_mini','swirr_log','swmini_log']]
dropfile_df=df_check

In [ ]:
#dropping the multiple duplicate due to the presence of Sw and Pc value.

#dropfile_df = dc_merge.drop_duplicates(subset='plug_id', keep='first')


In [ ]:
#Plot of log enty pressure (a_log) vs Porosity

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#dropfile_df['logkl'].values
palette = sns.color_palette("tab20", n_colors=263)
#sns.lmplot(dc_merge, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})

sns.scatterplot(dropfile_df, y="a_log", x="porosity" ,palette= palette,s=50 ,markers=True)
plt.xlabel('Porosity(frac)')
plt.ylabel('Entry Pressure')
plt.title(f'Entry Pressure vs Porosity ')
plt.ylim(0.01,1)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
#plt.savefig('pccurveimplot.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()



In [ ]:
#Irreducible water saturation vs Porosity

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
palette = sns.color_palette("tab20", n_colors=263)
#sns.lmplot(dc_merge, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})

sns.scatterplot(dropfile_df, y="swirr", x="porosity" ,palette= "magma",s=50 ,markers=True)
plt.ylabel('Swirr')
plt.xlabel('porosity')
plt.title(f'Irreducible water saturation vs Porosity ')
plt.ylim(0.003,0.6)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
#plt.savefig('pccurveimplot.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()


In [ ]:
#plot of permeability vs Porosity

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#sns.lmplot(dc_merge, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})
sns.scatterplot(dropfile_df, y="logkl", x="porosity" ,palette= "magma",s=50 ,markers=True)
#sns.scatterplot(dropfile_df, y="logkl", x="porosity" ,hue='RQI',palette= "magma",s=50 ,markers=True)
plt.xlabel('Porosity(frac)')
plt.ylabel('Log permeabilty(md)')
plt.title(f'Permeability vs Porosity ')
#plt.ylim(0.024,5)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
plt.savefig('permeabiltyporo.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:

#Plot of Log permeability vs porosity with log of entry pressure

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r


sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#sns.lmplot(dropfile_df, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})
c_value= dropfile_df['c_log'].values
c_value[ c_value<-5 ]=-5
plt.scatter( dropfile_df['porosity'].values, dropfile_df['logkl'].values, c=c_value , cmap= 'magma')
plt.xlabel('Porosity(frac)')
plt.ylabel('Permeabilty(mD - log)')
plt.title(f'Permeability vs Porosity with Entry Pressure ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
plt.legend()
plt.grid()
plt.savefig('entrypressureandpermebity.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:

#Plot of Log permeability vs porosity with log of relative pore size distribution pressure

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#sns.lmplot(dropfile_df, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})
a_value= dropfile_df['a_log'].values
a_value[ c_value<-5 ]=-5
plt.scatter( dropfile_df['porosity'].values, dropfile_df['logkl'].values, c=c_value , cmap= 'magma')
plt.xlabel('Porosity(frac)')
plt.ylabel('Permeabilty(mD - log)')
plt.title(f'Permeability vs Porosity with Entry Pressure ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
#plt.legend( labels)
plt.grid()
plt.savefig('entrypressureandpermebity.png')
#plt.yscale('log')
#plt.xscale('log')
plt.show()



# Principal component Analysis


In [ ]:
df_dropfile=df_check

In [ ]:
#BOXPLOT for selected variables.

dropfile_df
df_checker=dropfile_df[['logkl','porosity','swirr', 'a_log', 'c_log']]
len(df_checker.columns)


#check for outliers usings Box plot
plt.figure(figsize=(8,6))

for i, j in enumerate(df_checker.columns):
    plt.subplot(2,4,i+1)
    sns.boxplot(y=j, data=df_checker)
    plt.title("Boxplot of "+j)
    plt.tight_layout()
    plt.savefig('Box plot.png')
    
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.pairplot(df_checker)
plt.show()


In [ ]:
#Kaiser-Meyer-Olkin (KMO) Test: To check if we can use PCA in the variables or not.

kmo_all,kmo_model=calculate_kmo(df_checker)
kmo_model

In [ ]:
#heat map of the selected variables.

plt.figure(figsize =(10,6))
df_checker.corr()
sns.heatmap(df_checker.corr(),annot=True, cmap= "icefire")
plt.show()

In [ ]:
df_checker.describe().T

In [ ]:
#Normalization or scale the data

scaled_b=df_checker.apply(zscore)
scaled_b.head()
scaled_b

In [ ]:
#check for correlation from the standardized data 

plt.figure(figsize=(8,6))
sns.heatmap(scaled_b.corr(), annot=True, fmt='.2f')
plt.savefig('correlation of PCAscaled value.png')
plt.show()

In [ ]:
# perform PCA on the scaled data

pca = PCA(n_components=3)
new_pca_df = pca.fit_transform(scaled_b)
pc1, pc2 = new_pca_df [:, 0], new_pca_df [:, 1]
#scaled_data.shape
#x_pca.shape
print('Shape before PCA: ', scaled_b.shape)
print('Shape after PCA: ', new_pca_df.shape)
 
new_pca_df = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2', 'PC3'], index=scaled_b.columns)

print("PCA component shape: {}". format(pca.components_.shape))
print("PCA components:\n{}".format(pca.components_))
print(pca.explained_variance_)
print(pca.explained_variance_ratio_)
new_pca_df 

In [ ]:
''''''''''''''''''''''
# Calculate explained variance ratio and cumulative variance
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio)

# Plot the explained variance (bar plot) and cumulative variance (line plot)
plt.figure(figsize=(10, 6))

# Bar plot for explained variance ratio
plt.bar(range(1, len(explained_variance_ratio) + 1), 
        explained_variance_ratio * 100, 
        alpha=0.7, align='center', label='Individual Explained Variance (%)')

# Line plot for cumulative explained variance
plt.step(range(1, len(cumulative_variance) + 1), 
         cumulative_variance * 100, 
         where='post', linestyle='--', color='red', label='Cumulative Explained Variance (%)')

# Add percentage annotations on bars
for i, v in enumerate(explained_variance_ratio * 100):
    plt.text(i + 1, v + 1, f"{v:.1f}%", ha='center', va='bottom', fontsize=10)

# Labels and legend
plt.title('Explained Variance and Cumulative Variance by Principal Component')
plt.xlabel('Principal Component')
plt.ylabel('Percentage of Explained Variance')
plt.xticks(range(1, len(explained_variance_ratio) + 1))
plt.legend(loc='best')
plt.grid(alpha=0.5)

# Show plot
plt.tight_layout()
plt.show()


In [ ]:
#scree or elblow plot

plt.figure(figsize=(8,6))

plt.bar(range(1, len(pca.explained_variance_) + 1), pca.explained_variance_ratio_, label='Explained Variance', alpha=0.7)
plt.plot(range(1, len(pca.explained_variance_) + 1), np.cumsum(pca.explained_variance_ratio_), marker='o', linestyle='-', label='Cumulative Explained Variance', color='orange')

plt.xlabel('Principal Component')
plt.ylabel('Variance')
plt.title('Scree Plot: Explained Variance vs. Number of Principal Components')
plt.legend()
plt.grid()

# Add percentages as text labels to the bars
for i, exp_var in enumerate(pca.explained_variance_ratio_):
    plt.text(i + 1, exp_var, f'{exp_var * 100:.2f}%', ha='center', va='bottom')
plt.savefig('screeplotuse1.png')
plt.show()


In [ ]:
#scree plot
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Assuming scaled_b is your scaled dataset
pca = PCA()
pca.fit(scaled_b)

# Explained variance for each component (eigenvalues)
variance = pca.explained_variance_ratio_

# Plotting the Scree plot
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(variance) + 1), variance, marker='o', linestyle='--')
plt.title('Scree Plot')
plt.xlabel('Principal Components')
plt.ylabel('Variance')
plt.xticks(range(1, len(variance) + 1))
plt.savefig('screeplotuse2.png')
plt.show()

In [ ]:
# the principal components

selected_pca=new_pca_df[['PC1', 'PC2','PC3']]
selected_pca

In [ ]:
# The PCA Correlation
plt.figure(figsize=(8,6))
sns.heatmap(abs(selected_pca), annot=True, cmap='Blues', fmt='.2f')
plt.savefig('pcacorroletaionwith.png')

In [ ]:
scaled_b.iloc[0]

In [ ]:
for i in selected_pca.columns:
    pc_score=np.dot(selected_pca[i], scaled_b.iloc[0])
    print(round(pc_score,6),end='')

In [ ]:
#PCA Correlation
sns.heatmap(selected_pca.corr(), annot=True, fmt='.2f')
plt.savefig('pcacorroletaion123eachother.png')
plt.show()

In [ ]:
# Biplot for the PCA in 3D
import matplotlib.cm as cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Step 1: Loading and Standardization of Data

datapca_df = scaled_b

# Standardize the data
#scaler = StandardScaler()
scaled_data = datapca_df

# Converting scaled_data to a DataFrame to retain column names
scaled_data_df = pd.DataFrame(scaled_data, columns=datapca_df.columns)

# Step 2: Perform PCA for the ComponentsØ meanwhile you can determine the number of componnent
pca = PCA(n_components=3)  # Use n principal components
pca_components = pca.fit_transform(scaled_data)

# Step 3: Create a 3D Biplot
def biplot_3d(datapca_df, pca_model):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Plot the observations in 3D space
    ax.scatter(datapca_df[:, 0], datapca_df[:, 1], datapca_df[:, 2], alpha=0.8)

    # Plot the vectors (loadings)
    for i, (pc1_loading, pc2_loading, pc3_loading) in enumerate(pca_model.components_.T):
        ax.quiver(0, 0, 0, pc1_loading, pc2_loading, pc3_loading, color='green', alpha=0.6)
        ax.text(pc1_loading * 1.15, pc2_loading * 1.15, pc3_loading * 1.15, 
                scaled_data_df.columns[i], color='black', ha='center', va='center')

    # Set labels and title
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    ax.set_title('3D Biplot of Principal Components')
    plt.savefig('PCcompnents.png')
    plt.show()

# Step 4: Call the Biplot Function
biplot_3d(pca_components, pca)


In [ ]:

# plot of PCA in 2D

def biplot_2d(pc_x, pc_y, pca_model, component_labels):
    plt.figure(figsize=(10, 7))
    
    # Plot the observations in 2D space
    plt.scatter(pc_x, pc_y, alpha=0.8, edgecolors='k')
    
    # Plot the vectors (loadings)
    for i, (pc_x_loading, pc_y_loading) in enumerate(pca_model.components_[:2].T):
        plt.arrow(0, 0, pc_x_loading, pc_y_loading, color='green', alpha=0.9, head_width=0.05)
        plt.text(pc_x_loading * 1.16, pc_y_loading * 1.16, scaled_data_df.columns[i], 
                 color='red', ha='center', va='center')
    
    plt.xlabel(component_labels[0])
    plt.ylabel(component_labels[1])
    plt.title(f' {component_labels[0]} vs {component_labels[1]}')
    plt.grid(visible=True, linestyle='--', alpha=0.4)
    plt.savefig('PC2andPC3forpca')
    plt.show()

#Plot PC1 vs PC2
biplot_2d(pca_components[:, 0], pca_components[:, 1], pca, ['Principal Component 1', 'Principal Component 2'])

#Plot PC2 vs PC3
biplot_2d(pca_components[:, 1], pca_components[:, 2], pca, ['Principal Component 2', 'Principal Component 3'])


In [ ]:
#PCA Loadings


def loading_plot(pca_model, feature_names):
    plt.figure(figsize=(8, 6))
    
    # Plot the loadings for each variable on PC1 and PC2
    for i, (pc1_loading, pc2_loading) in enumerate(pca_model.components_[:2].T):
        plt.scatter(pc1_loading, pc2_loading, alpha=0.9)
        plt.text(pc1_loading, pc2_loading, feature_names[i], ha='right', va='bottom', fontsize=11)
    
    plt.xlabel('Principal Component 1 Loading')
    plt.ylabel('Principal Component 2 Loading')
    plt.title('PC1 vs PC2 Loading Plot')
    plt.axhline(0, color='black', linestyle='--', alpha=0.7)
    plt.axvline(0, color='black', linestyle='--', alpha=0.7)
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    plt.savefig('loadingplot1and3')
    plt.show()

# Call the loading plot function
loading_plot(pca, scaled_data_df.columns)

In [ ]:
# Loading Plot for PC2 and PC3

def loading_plot_pc2_pc3(pca_model, feature_names):
    plt.figure(figsize=(8, 6))
    
    # Plot the loadings for each variable on PC2 and PC3
    for i, (pc2_loading, pc3_loading) in enumerate(pca_model.components_[1:3].T):
        plt.scatter(pc2_loading, pc3_loading, alpha=0.9)
        plt.text(pc2_loading, pc3_loading, feature_names[i], ha='right', va='bottom', fontsize=11)
    
    plt.xlabel('Principal Component 2 Loading ')
    plt.ylabel('Principal Component 3 Loading ')
    plt.title(' PC2 vs PC3 Loading Plot')
    plt.axhline(0, color='black', linestyle='--', alpha=0.7)
    plt.axvline(0, color='black', linestyle='--', alpha=0.7)
    plt.grid(visible=True, linestyle='--', alpha=0.5)
    plt.savefig('loadingplot2and3')
    plt.show()

# Call the loading plot function for PC2 and PC3
loading_plot_pc2_pc3(pca, scaled_data_df.columns)



## K- MEANS ALGORITHM #

In [ ]:
#Dataset for the K means
scaled_b

In [ ]:
'''''''''

#load data and normalise or scale data
df_kmeans=df_checker
scaler=StandardScaler()
norm_data=scaler.fit_transform(df_kmeans)
norm_data
# choose you number of cluster
num_cluster=3
k_load=KMeans(num_cluster, n_init=12)
k_load.fit_predict(norm_data)
labels=k_load.labels_
print(labels)
labels.shape

dc_merge
norm_data
df_kmeans["features"]=labels
df_kmeans

In [ ]:
# Perform k-means 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Step 1: Load and Standardize Your Data
datapca_k = scaled_b
#imported from our already scaled data
scaled_data = datapca_k

# Step 2: Perform K-Means Clustering
# Choose the number of clusters
k = 3  # choose different numbers of clusters you like.

# K-Means
kmeans = KMeans(n_clusters=k, random_state=0)
clusters = kmeans.fit_predict(scaled_data)

# Add the cluster labels to the DataFrame
datapca_k.loc[:, 'Cluster'] = clusters
datapca_k



In [ ]:
#only if you wish to visualise the already clustered on PCA
''''''''''''''''

# Step 3: Visualize the Clusters Using PCA

pca = PCA(n_components=3)
pca_components = pca.fit_transform(scaled_data)

# Plot the clusters
plt.figure(figsize=(10, 7))
for cluster in range(k):
    plt.scatter(
        pca_components[clusters == cluster, 0], 
        pca_components[clusters == cluster, 1], 
        label=f'Cluster {cluster}'
    )
    
# Plot the cluster centroids
centroids = kmeans.cluster_centers_
centroids_pca = pca.transform(centroids)  # Transform centroids using PCA
plt.scatter(centroids_pca[:, 0], centroids_pca[:, 1], c='black', marker='X', s=100, label='Centroids')

plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('K-Means Clustering with PCA')
plt.legend()
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:

''''''''''''''''
# Step 3: Perform PCA for Three Components to view on 3d
pca_3d = PCA(n_components=3)
pca_3d_components = pca_3d.fit_transform(scaled_data)

# Step 4: Visualize K-Means Clustering Using PCA Components 1, 2, and 3
def plot_kmeans_pca_3d(pca_components, clusters, k):
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the clustered data points
    for cluster in range(k):
        ax.scatter(
            pca_components[clusters == cluster, 0], 
            pca_components[clusters == cluster, 1], 
            pca_components[clusters == cluster, 2],
            label=f'Cluster {cluster}', alpha=0.7
        )
    
    # Plot the cluster centroids
    centroids_pca = pca_3d.transform(kmeans.cluster_centers_)  # Transform centroids using PCA
    ax.scatter(
        centroids_pca[:, 0], centroids_pca[:, 1], centroids_pca[:, 2], 
        c='black', marker='X', s=100, label='Centroids'
    )

    # Set axis labels
    ax.set_xlabel('Principal Component 1', labelpad=10)
    ax.set_ylabel('Principal Component 2', labelpad=10)
    ax.set_zlabel('Principal Component 3', labelpad=10)

    ax.set_title('K-Means Clustering with PCA (PC1, PC2, PC3)')
    ax.legend()
    plt.show()

# Call the function to plot K-Means clustering in 3D using PCA components 1, 2, and 3
plot_kmeans_pca_3d(pca_3d_components, clusters, k)

In [ ]:
## K MEANS ALGORITHM

In [ ]:
#Plot of Kmeans for the the principal componenets
plt.figure(figsize=(10,6))
#sns.scatterplot(data=b, x="sw",y="pc", hue="features", palette="coolwarm")
#sns.lineplot(data=b, x="sw",y="pc", hue="features", palette="coolwarm")
 #ax.scatter(datapca_df[:, 0], datapca_df[:, 1], datapca_df[:, 2], alpha=0.8)
mglearn.discrete_scatter(pca_components[:,0],pca_components[:,1],datapca_k['Cluster'])
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title(f'K-MEANS RESULT')
plt.legend()
#plt.xlim(0,7)
plt.grid()
plt.savefig('aaaKmeans1')
#plt.yscale('log')
plt.show()

In [ ]:

plt.figure(figsize=(8,6))
#sns.scatterplot(data=b, x="sw",y="pc", hue="features", palette="coolwarm")
#sns.lineplot(data=b, x="sw",y="pc", hue="features", palette="coolwarm")
 #ax.scatter(datapca_df[:, 0], datapca_df[:, 1], datapca_df[:, 2], alpha=0.8)
mglearn.discrete_scatter(pca_components[:,1],pca_components[:,2],datapca_k['Cluster'])
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title(f'K-MEANS RESULT')
plt.legend()
#plt.xlim(0,7)
plt.grid()
plt.savefig('aaaKmeans1')
#plt.yscale('log')
plt.show()

In [ ]:
dc_merge.describe()

# Plotting of capillary presure according to their clusters

In [ ]:
plt.figure(figsize=(8,6))
#sns.scatterplot(data=a, x="sw",y="pc", hue="features", palette="coolwarm")
#sns.lineplot(data=a, x="sw",y="pc", hue="features", palette="coolwarm")
mglearn.discrete_scatter(dc_merge['scaled_sw'], dc_merge['pc'],datapca_k['Cluster'],s=7)
plt.xlabel('Water saturation (Sw)')
plt.ylabel('Capillary pressure(Pc)')
plt.title(f'Classification Result')
plt.legend()
#plt.xlim(0,7)
plt.grid()
plt.savefig('12Kmeans_result.png')
#plt.yscale('log')
plt.show()

In [ ]:
#Add dataframe containing the cluster to the general Dataframe
datapca_k['Cluster']
new_features=datapca_k['Cluster']
new_features
dc_merge['cluster']=datapca_k['Cluster']
print(dc_merge)
df_cluster0=dc_merge[dc_merge['cluster']==0]
df_cluster1=dc_merge[dc_merge['cluster']==1]
df_cluster2=dc_merge[dc_merge['cluster']==2]
print("Rows with role 0:")
print(df_cluster0)

In [ ]:
# For all clusters
dl = df_cluster0
dv = df_cluster1
ds = df_cluster2

#figure
plt.figure(figsize=(12, 8))

# Loop through each cluster DataFrame and each unique plug_id
for df, color, label in zip([dl, dv, ds], ['blue', 'brown', 'green'], ['Rocktype 0', 'Rocktype 1', 'Rocktype 2']):
    for plug_id in df['plug_id'].unique():
        subset = df[df['plug_id'] == plug_id]
        sns.lineplot(x=subset['scaled_sw'], y=subset['pc'], label=label if plug_id == df['plug_id'].unique()[0] else "", color=color)

# Customize the plot
plt.title('Plot of the Three rock type')
plt.xlabel('Water Saturation (frac)')
plt.ylabel('Capillary pressure (Pc, Psi)')
plt.legend(title='Rock Types')
#plt.xlim(0,7)
plt.grid()
plt.savefig('Allcluster_result.png')
#plt.yscale('log')
plt.show()          



In [ ]:
#For cluster 2

dl = df_cluster2

plt.figure(figsize=(10,6))


for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['pc'], c='green')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('Water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc, Psi)')
plt.title('Rock type 2')
plt.grid()
#plt.yscale('log')
plt.ylim(0,6)
#plt.xlim(0, 1.05)
plt.savefig('Rockty2.png')
#plt.yscale('log')
plt.show()


In [ ]:
# for cluster 1
dl = df_cluster1

plt.figure(figsize=(10,6))
for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['pc'], c='brown')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('Water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc, Psi)')
plt.title('Rock type 1')
plt.grid()
#plt.yscale('log')
plt.ylim(0,6)
#plt.xlim(0, 1.05)
plt.savefig('Rocktyoe1.png')
#plt.yscale('log')
plt.show()

In [ ]:
# for cluster 
dl = df_cluster0

plt.figure(figsize=(10,6))
for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['pc'], c='blue')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('Water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc, Psi)')
plt.title('Rock type 0')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
#plt.xlim(0, 1.05)
plt.savefig('Rocktyp0.png')
#plt.yscale('log')
plt.show()

## J-function Curves for eaxch clusters

In [ ]:
dl = df_cluster0

plt.figure(figsize=(10,6))
for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['J_function'], c='blue')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc)')
plt.title('J-function for Cluster0')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
#plt.xlim(0, 1.05)
#plt.savefig('Firstccapillarycurve.png')
#plt.yscale('log')
plt.show()


In [ ]:
dl = df_cluster1

plt.figure(figsize=(10,6))
for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['J_function'], c='blue')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc)')
plt.title('J-function for Cluster1')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
#plt.xlim(0, 1.05)
#plt.savefig('Firstccapillarycurve.png')
#plt.yscale('log')
plt.show()

In [ ]:
dl = df_cluster2

plt.figure(figsize=(10,6))
for plug_id in dl['plug_id'].unique():
    subset = dl[dl['plug_id'] == plug_id]
    plt.plot(subset['scaled_sw'], subset['J_function'], c='blue')

# plot of Capillary pressure against Water Saturation 

plt.xlabel('water saturation(Sw)')
plt.ylabel('Capillary Pressure(Pc)')
plt.title('J-function for Cluster2')
plt.grid()
#plt.yscale('log')
#plt.ylim(0,16)
#plt.xlim(0, 1.05)
#plt.savefig('Firstccapillarycurve.png')
#plt.yscale('log')
plt.show()

In [ ]:
dc_merge

In [ ]:
dl = df_cluster0
dv = df_cluster1
ds = df_cluster2

#figure
plt.figure(figsize=(10, 6))

# Loop through each cluster DataFrame and each unique plug_id
for df, color, label in zip([dl, dv, ds], ['blue', 'brown', 'green'], ['Rocktype 0', 'Rocktype 1', 'Rocktype 2']):
    for plug_id in df['plug_id'].unique():
        subset = df[df['plug_id'] == plug_id]
        sns.scatterplot(x=subset['Norm_porosity'], y=subset['logkl'] ,palette= "deep",s=90 ,label=label if plug_id == df['plug_id'].unique()[0] else "", color=color)

# Customize the plot

plt.xlabel('Porosity(frac)')
plt.ylabel('Permeabilty(mD - log)')
plt.title(f'Permeability vs porosity crossplot ')
plt.legend(title='Rock Types')
#plt.xlim(0,7)
plt.grid()
plt.savefig('poropermrocktye.png')
plt.show()    



In [ ]:

# Peemeability and porossity for ALL CLUSTERS

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#sns.lmplot(dc_merge, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})
sns.scatterplot(dc_merge, y="kl", x="Norm_porosity" ,hue='cluster',style='cluster',palette= "deep",s=90)
#sns.scatterplot(df_cluster1, y="logkl", x="porosity" ,hue='cluster',palette= "viridis",s=50 ,markers=True)
#sns.scatterplot(df_cluster2, y="logkl", x="porosity" ,hue='cluster',palette= "flare",s=50 ,markers=True)
plt.xlabel('Porosity(frac)')
plt.ylabel('Permeabilty(mD - log)')
plt.title(f'Permeability vs Porosity ')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
plt.savefig('kmeanonpermeporo.png')
plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:
# RQI FOR ALL CLUSTERS

#ranking=["g1", "g2", "g3", "g4"], palette: flare, crest, magma, viridis, rocket_r
sns.set_theme(style="ticks")
plt.figure(figsize=(10,6))
#sns.lmplot(dc_merge, x="porosity", y="kl" , lowess=True, line_kws={"color": "C1"})

sns.scatterplot(df_cluster0, y="RQI", x="Norm_porosity" ,hue='cluster',palette= "viridis",s=50 ,markers=True)
sns.scatterplot(df_cluster1, y="RQI", x="Norm_porosity" ,hue='cluster',palette= "magma",s=50 ,markers=True)
sns.scatterplot(df_cluster2, y="RQI", x="Norm_porosity" ,hue='cluster',palette= "viridis",s=50 ,markers=True)
plt.xlabel('Normalised Porosity(frac)')
plt.ylabel('Rock Quality Index(RQI)')
plt.title(f'RQI vs Normalised Porosity')
#plt.ylim(0.01,10)
#plt.xlim(0.1,1)
#plt.legend()
plt.grid()
plt.savefig('RQIonNorm1.png')
plt.yscale('log')
#plt.xscale('log')
plt.show()

In [ ]:
df_cluster0

In [ ]:
'''''
sns.histplot(dc_merge, x='kl', bins=20, kde=True, color='blue')
plt.title('Histogram of Permeability')
plt.xlabel('Permeability')
plt.ylabel('Frequency')
plt.show()

# Permeability Distribution Analysis 

In [ ]:
# Permeability Distribution Analysis 

sns.set(style='whitegrid')

# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# 1.
sns.histplot(data=df_cluster0, x='kl', bins=30, kde=True, color='blue', ax=axes[0, 0])
axes[0, 0].set_title('Histogram and KDE Permeabilty, Cluster 0')

# 2. 
sns.histplot(data=df_cluster1, x='kl', bins=30, kde=True, color='brown', ax=axes[0, 1])
axes[0, 1].set_title('Histogram and KDE Permeabilty,  Cluster 1')

# 3. 
sns.histplot(data=df_cluster2, x='kl', bins=30, kde=True, color='green', ax=axes[1, 0])
axes[1, 0].set_title('Histogram and KDE Permeabilty,  Cluster 2')


plt.tight_layout()
plt.savefig('histocluster')
plt.show()

In [ ]:
# PORSOSITY Distribution Analysis 

sns.set(style='whitegrid')

# Create a figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# 1.
sns.histplot(data=df_cluster0, x='porosity', bins=30, kde=True, color='blue', ax=axes[0, 0])
axes[0, 0].set_title('Histogram and KDE Porosity, Cluster 0')

# 2. 
sns.histplot(data=df_cluster1, x='porosity', bins=30, kde=True, color='brown', ax=axes[0, 1])
axes[0, 1].set_title('Histogram and KDE Porosity,  Cluster 1')

# 3. 
sns.histplot(data=df_cluster2, x='porosity', bins=30, kde=True, color='green', ax=axes[1, 0])
axes[1, 0].set_title('Histogram and KDE Porosity,  Cluster 2')


plt.tight_layout()
plt.savefig('histoclusterporo')
plt.show()

# Prediction of C and A with supervised learning (ANN)

In [ ]:
import pandas as pd
import numpy as np
from io import StringIO
from scipy.optimize import curve_fit
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.layers import Input, Dense

In [ ]:
dc_merge

In [ ]:
''''
df_clean=dc_merge

X = df_clean[['kl', 'porosity', 'swirr']].values
y = df_clean[['c', 'a']].values

X_train, X_test, y_train,  y_test= train_test_split(
    X, y, test_size=0.2, random_state=0
)
print(X_test.shape, X_train.shape)


X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [ ]:
df_clean=dc_merge.copy()


# Features and targets
X = df_clean[['logkl', 'porosity', 'swirr']].values
y = df_clean[['c_log', 'a_log']].values
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)
print(X_test.shape, X_train.shape)



In [ ]:


# Feature scaling
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)


# Define neural network architecture

    #Dense(2, activation='linear')  # Output layer for c and a
model = Sequential()
model.add(Input(shape=(3,)))
model.add(Dense(units=10, activation='relu'))
#model.add(Dense(units=12, activation='relu'))
model.add(Dense(units=10, activation='relu'))
# Output layer for c and a
model.add(Dense(units=2, activation='linear')) 
#])

# Compiling the model
#model.compile(optimizer='adam', loss='mse', metrics=['mae'])
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Define early stopping to prevent overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Training the model usingg fit
history = model.fit(
    X_train_scaled, y_train_scaled,
    epochs=150,
    batch_size=36,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=1  #  0 to hide training progress
)

# Evaluate the model on the test set
test_loss, test_mae = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"\nTest Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

print("Neural Network Architecture:")
model.summary()

# Plotting the Learning Curves
def plot_learning_curves(history):
 
    # Extract data from history
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    mae = history.history['mae']
    val_mae = history.history['val_mae']
    
    epochs = range(1, len(loss) + 1)
    
    # Plot Loss (MSE)
    plt.figure(figsize=(14, 6))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Training Loss (MSE)')
    plt.plot(epochs, val_loss, 'ro-', label='Validation Loss (MSE)')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Squared Error')
    plt.savefig('ANN MSE')
    plt.legend()
    plt.grid(True)
    
    # Plot MAE
    plt.subplot(1, 2, 2)
    plt.plot(epochs, mae, 'bo-', label='Training MAE')
    plt.plot(epochs, val_mae, 'ro-', label='Validation MAE')
    plt.title('Training and Validation MAE')
    plt.xlabel('Epochs')
    plt.ylabel('Mean Absolute Error')
    plt.legend()
    plt.grid(True)
    plt.savefig('ANn MAE.png')
    
    plt.tight_layout()
    plt.show()

# Call the plotting function
plot_learning_curves(history)


#  Predictions and Plotting
# Predict c and a for the dataset
X_scaled = scaler_X.transform(df_clean[['logkl', 'porosity', 'swirr']].values)
y_pred_scaled = model.predict(X_scaled)
y_pred = scaler_y.inverse_transform(y_pred_scaled)

# Add predictions to the DataFrame
df_clean['c*'] = y_pred[:, 0]
df_clean['a*'] = y_pred[:, 1]

print("\nData with Predicted Parameters (c* and a*):")
print(df_clean[['plug_id', 'kl', 'porosity', 'swirr', 'c', 'a', 'c*', 'a*']].head())


In [ ]:
#df_clean
df_clean1=df_clean[['plug_id','pc','sw', 'swirr' , 'c_log' ,'a_log' ,'kl','logkl', 'c','a', 'c*', 'a*']]
df_clean1.dropna()

In [ ]:

df_clean2 = df_clean1.copy()

df_clean2['c_un*'] = 10**(df_clean2['c*'])  
df_clean2['a_un*'] = 10**(df_clean2['a*'])  
df_clean2
#df['log_c*'] = np.log10(df_clean2['c*'])
#df['log_a*'] = np.log10(df_clean2['a*'])



## Just for sample


In [ ]:
# Function to compute capillary pressure using Skjaeveland model
def compute_pc(sw, c, a, swirr):
    # Avoid division by zero and invalid sw values
    with np.errstate(divide='ignore', invalid='ignore'):
        denominator = ((sw - swirr) / (1 - swirr)) ** a
        return c / denominator

# Plotting for a sample plug_id
sample_plug_id = df_clean2['plug_id'].iloc[0]

# Filter data for the sample plu_id
sample_data = df_clean2[df_clean2['plug_id'] == sample_plug_id]




plt.figure(figsize=(10, 6))
plt.scatter(sample_data['sw'], sample_data['pc'], color='blue', label='Observed pc')

# Compute pc using original parameters
sw_values = np.linspace(sample_data['swirr'].iloc[0] + 0.01, sample_data['sw'].max(), 100)
pc_original = compute_pc(sw_values, sample_data['c'].iloc[0], sample_data['a'].iloc[0], sample_data['swirr'].iloc[0])
plt.plot(sw_values, pc_original, color='green', label='Fitted pc (Original c & a)')


pc_predicted = compute_pc(sw_values, sample_data['c*'].iloc[0], sample_data['a*'].iloc[0], sample_data['swirr'].iloc[0])
plt.plot(sw_values, pc_predicted, color='red', linestyle='--', label='Fitted pc (Predicted c* & a*)')

plt.title(f'Capillary Pressure Curve for plug_id {sample_plug_id}')
plt.xlabel('Water Saturation (sw)')
plt.ylabel('Capillary Pressure (pc)')
plt.legend()
plt.grid(True)
plt.show()



## Visualization 

In [ ]:

# Function to compute capillary pressure using Skjaeveland model
def compute_pc(sw, c, a, swirr):
    # Avoid division by zero and invalid sw values
    with np.errstate(divide='ignore', invalid='ignore'):
        d_min = ((sw - swirr) / (1 - swirr)) ** a
        return c / d_min

# Create a directory to save the plots
plots_dir = 'capillary_pressure_plots'
os.makedirs(plots_dir, exist_ok=True)

# Get all unique plu_id values
unique_plug_ids = df_clean2['plug_id'].unique()

# Loop through each plu_id and plot
for plug_id in unique_plug_ids:
    # Filter data for the current plug_id
    plu_data = df_clean2[df_clean2['plug_id'] == plug_id].copy()
    
    # Ensure the data is sorted by sw for smooth curves
    plu_data.sort_values(by='sw', inplace=True)
    
    # Observed sw and pc
    sw_observed = plu_data['sw'].values
    pc_observed = plu_data['pc'].values
    
    # Parameters
    c_original = plu_data['c'].iloc[0]
    a_original = plu_data['a'].iloc[0]
    c_predicted = plu_data['c*'].iloc[0]
    a_predicted = plu_data['a*'].iloc[0]
    swirr = plu_data['swirr'].iloc[0]
    
    # Generate sw values for plotting fitted curves
    sw_min = np.min(sw_observed)  
    sw_max = sw_observed.max()
    sw_fitted = np.linspace(sw_min, sw_max, 100)
    
    # Compute capillary pressure using original parameters
    pc_original_fitted = compute_pc(sw_fitted, c_original, a_original, swirr)
    
    # Compute capillary pressure using predicted parameters
    pc_predicted_fitted = compute_pc(sw_fitted, c_predicted, a_predicted, swirr)
    
    # Compute predicted pc for observed sw to calculate R²
    pc_predicted_observed = compute_pc(sw_observed, c_predicted, a_predicted, swirr)
    
    # Handle potential NaN or infinite values in pc_predicted_observed
    valid_indices = ~np.isnan(pc_predicted_observed) & ~np.isinf(pc_predicted_observed)
    if np.sum(valid_indices) < 2:
        r2 = np.nan  # Not enough points to compute R²
    else:
        r2 = r2_score(pc_observed[valid_indices], pc_predicted_observed[valid_indices])
    
    # Plotting
    plt.figure(figsize=(10, 6))
    plt.scatter(sw_observed, pc_observed, color='blue', label='Observed pc')
    plt.plot(sw_fitted, pc_original_fitted, color='green', label='Fitted pc (Original c & a)')
    plt.plot(sw_fitted, pc_predicted_fitted, color='red', linestyle='--', label='Fitted pc (Predicted c_un* & c_un*)')
    
    plt.title(f'Capillary Pressure Curve for plug_id {plug_id}\nR² = {r2:.4f}')
    plt.xlabel('Water saturation (sw)')
    plt.ylabel('Capillary Pressure (pc)')
    plt.legend()
    plt.grid(True)
    plt.savefig('modeelingforpc1NORSQUARE.png')
    plt.show()
    # Save the plot
    #plt.savefig(os.path.join(plots_dir, f'capillary_pressure_plu_{plug_id}.png'))
    #plt.close()
   

#print(f"\nCapillary pressure plots saved in the '{plots_dir}' directory.")

# Visualize Model Performance


# Scatter plot of actual vs predicted c
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['c'], df_clean2['c*'], color='purple', alpha=0.6)
#plt.plot([df_clean['c'].min(), df_clean['c'].max()], [df_clean['c'].min(), df_clean['c'].max()], 'k--', lw=2)
plt.xlabel('Actual c')
plt.ylabel('Predicted c*')
plt.title('Actual vs Predicted c')
plt.grid(True)
plt.show()

# Scatter plot of actual vs predicted a
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['a'], df_clean2['a*'], color='orange', alpha=0.6)
#plt.plot([df_clean['a'].min(), df_clean['a'].max()], [df_clean['a'].min(), df_clean['a'].max()], 'k--', lw=2)
plt.xlabel('Actual a')
plt.ylabel('Predicted a*')
#plt.ylog(0,10)
plt.title('Actual vs Predicted a')
plt.grid(True)
plt.show()


In [ ]:
# Scatter plot of actual vs predicted c
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['c'], df_clean2['c*'], color='purple', alpha=0.6)
plt.plot([-1,3],[-1,3],'k--',lw=2)
#plt.plot([df_clean['c'].min(), df_clean['c'].max()], [df_clean['c'].min(), df_clean['c'].max()], 'k--', lw=2)
plt.xlabel('Actual c')
plt.ylabel('Predicted c*')
plt.title('Actual vs Predicted c')
plt.grid(True)
plt.show()

# Scatter plot of actual vs predicted a
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['a'], df_clean2['a*'], color='orange', alpha=0.6)
plt.plot([-1,5],[-1,5],'k--',lw=2)
#plt.plot([df_clean['a'].min(), df_clean['a'].max()], [df_clean['a'].min(), df_clean['a'].max()], 'k--', lw=2)
plt.xlabel('Actual a')
plt.ylabel('Predicted a*')
#plt.ylog(0,10)
plt.title('Actual vs Predicted a')
plt.grid(True)
plt.show()

In [ ]:


# compute capillary pressure using Skjaeveland model
def compute_pc(sw, c, a, swirr):
    # Avoid division by zero and invalid sw values
    with np.errstate(divide='ignore', invalid='ignore'):
        denominator = ((sw - swirr) / (1 - swirr)) ** a
        return c / denominator

# Get all unique plu_id values
unique_plug_ids = df_clean['plug_id'].unique()

# Initialize a list to keep track of plug_ids with R² >= 0.5
plug_ids_with_high_r2 = []

# Initialize a list to keep track of plug_ids with R² < 0.5
plug_ids_with_low_r2 = []

# Loop through each plu_id and plot if R² >= 0.5
for plug_id in unique_plug_ids:
    # Filter data for the current plug_id
    plu_data = df_clean2[df_clean2['plug_id'] == plug_id].copy()
    
    # Ensure the data is sorted by sw for smooth curves
    plu_data.sort_values(by='sw', inplace=True)
    
    # Check if 'sw' and 'pc' columns exist
    if 'sw' not in plu_data.columns or 'pc' not in plu_data.columns:
        print(f"Skipping plug_id {plug_id}: Missing 'sw' or 'pc' columns.")
        continue
    
    # Observed sw and pc
    sw_observed = plu_data['sw'].values
    pc_observed = plu_data['pc'].values
    
    # Parameters
    c_original = plu_data['c'].iloc[0]
    a_original = plu_data['a'].iloc[0]
    c_predicted = plu_data['c*'].iloc[0]
    a_predicted = plu_data['a*'].iloc[0]
    swirr = plu_data['swirr'].iloc[0]
    
    # Generate sw values for plotting fitted curves
    sw_min = np.min(sw_observed)  
    sw_max = sw_observed.max()
    sw_fitted = np.linspace(sw_min, sw_max, 100)
    
    #  capillary pressure using original parameters
    pc_original_fitted = compute_pc(sw_fitted, c_original, a_original, swirr)
    
    #  capillary pressure using predicted parameters
    pc_predicted_fitted = compute_pc(sw_fitted, c_predicted, a_predicted, swirr)
    
    #  predicted pc for observed sw to calculate R²
    pc_predicted_observed = compute_pc(sw_observed, c_predicted, a_predicted, swirr)
    
    # Handle NaN or infinite values in pc_predicted_observed
    valid_indices = ~np.isnan(pc_predicted_observed) & ~np.isinf(pc_predicted_observed)
    if np.sum(valid_indices) < 2:
        r2 = np.nan  # Not enough points to compute R²
    else:
        r2 = r2_score(pc_observed[valid_indices], pc_predicted_observed[valid_indices])
    
    # Check if R² is greater than or equal to 0.5
    if r2 >= 0.5:
        plug_ids_with_high_r2.append(plug_id)
        
        # Plotting
        plt.figure(figsize=(10, 6))
        plt.scatter(sw_observed, pc_observed, color='blue', label='Observed pc')
        plt.plot(sw_fitted, pc_original_fitted, color='green', label='Fitted pc (Original c & a)')
        plt.plot(sw_fitted, pc_predicted_fitted, color='red', linestyle='--', label='Fitted pc (Predicted c_un* & a_un*)')
        
        plt.title(f'Capillary Pressure Curve for plug_id {plug_id}\nR² = {r2:.4f}')
        plt.xlabel('Water Saturation (sw)')
        plt.ylabel('Capillary Pressure (pc)')
        
        plt.legend()
        plt.grid(True)
        plt.savefig('modeelingforpc1withR.png')
        
        plt.show()
    else:
        plug_ids_with_low_r2.append(plug_id)

#  Results
print("\nSummary of Plots Displayed:")
print(f"Total unique plug_id values: {len(unique_plug_ids)}")
print(f"Plug_ids with R² >= 0.5: {len(plug_ids_with_high_r2)}")
print(f"Plug_ids with R² < 0.5: {len(plug_ids_with_low_r2)}")

if plug_ids_with_low_r2:
    print("\nPlug_ids with R² < 0.5 (Plots not displayed):")
    for pid in plug_ids_with_low_r2:
        print(f"- {pid}")



In [ ]:

 #Visualize Model Performance

# Scatter plot of actual vs predicted c
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['c_log'], df_clean2['c*'], color='purple', alpha=0.6)
plt.plot([-10,2],[-10,2],'k--',lw=2)
#plt.plot([df_clean1['c_log'].min(), df_clean1['c_log'].max()], [df_clean1['c*'].min(), df_clean1['c*'].max()], 'k--', lw=2)
plt.xlabel('Actual c (c_log)')
plt.ylabel('Predicted c*')
plt.ylim(-5,2)
plt.xlim(-5,2)
#plt.gca().set_aspect('equal', adjustable='box')
plt.title('Actual vs predicted entry pressure')
plt.savefig('ANN C.png')
plt.grid(True)
plt.show()




In [ ]:
# Scatter plot of actual vs predicted a
plt.figure(figsize=(8, 6))
plt.scatter(df_clean2['a_log'], df_clean['a*'], color='orange', alpha=0.6)
plt.plot([0,2],[0,2],'k--',lw=2)
#plt.plot([df_clean['a'].min(), df_clean['a'].max()], [df_clean['a'].min(), df_clean['a'].max()], 'k--', lw=2)
plt.xlabel('Actual a(a_log)')
plt.ylabel('Predicted a*')
plt.ylim(-0.5,3)
plt.xlim(-0.5,3)
#plt.gca().set_aspect('equal', adjustable='box')
plt.title('Actual vs predicted pores size distribution')
plt.savefig('ANN A.png')
plt.grid(True)
plt.show()

In [ ]:
df_clean2

In [ ]:
df_clean2
df_outlier=df_clean2[[ 'c_log','a_log', 'a*','c*', 'c_un*','a_un*']].copy()
len(df_outlier.columns)


#check for outliers usings Box plot
plt.figure(figsize=(8,6))

for i, j in enumerate(df_outlier.columns):
    plt.subplot(2,4,i+1)
    sns.boxplot(y=j, data=df_outlier)
    plt.title("Boxplot of "+j)
    plt.tight_layout()
    plt.savefig('ANNBox plot.png')
    
plt.show()